In [ ]:
!pip -q install roboflow ultralytics==8.0.196

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 80.2 MB/s eta 0:00:00


In [ ]:
import cv2
import os
from roboflow import Roboflow
from ultralytics import YOLO
import yaml

In [ ]:
rf = Roboflow(api_key="#########")
project = rf.workspace("universe-datasets").project("hard-hat-universe-0dy7t")
version = project.version(26)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Hard-Hat-Universe-26 in yolov8:: 100%|██████████| 14080/14080 [00:02<00:00, 5226.29it/s]


In [ ]:
assert len(
    os.listdir("Hard-Hat-Universe-26/train/images")
) == len(os.listdir("Hard-Hat-Universe-26/train/labels"))
assert len(
    os.listdir("Hard-Hat-Universe-26/valid/images")
) == len(os.listdir("Hard-Hat-Universe-26/valid/labels"))
assert len(
    os.listdir("Hard-Hat-Universe-26/test/images")
) == len(os.listdir("Hard-Hat-Universe-26/test/labels"))

print("Train size:", len(os.listdir("Hard-Hat-Universe-26/train/images")))
print("Valid size:", len(os.listdir("Hard-Hat-Universe-26/valid/images")))
print("Test size:", len(os.listdir("Hard-Hat-Universe-26/test/images")))

Train size: 4912
Valid size: 1414
Test size: 708


In [ ]:
f = open("Hard-Hat-Universe-26/README.dataset.txt", "r")
print(f.read())

# Hard Hat Universe > no_nulls_plain
https://universe.roboflow.com/universe-datasets/hard-hat-universe-0dy7t

Provided by a Roboflow user
License: Public Domain

# Overview

The `Hard Hat` dataset is an object detection dataset of workers in workplace settings that require a hard hat. Annotations also include examples of just "person" and "head," for when an individual may be present without a hard hart.

Example Image:
![Example Image](https://i.imgur.com/7spoIJT.png)

# Use Cases

One could use this dataset to, for example, build a classifier of workers that are abiding safety code within a workplace versus those that may not be. It is also a good general dataset for practice.

# Using this Dataset

Use the `fork` button to copy this dataset to your own Roboflow account and export it with new preprocessing settings (perhaps resized for your model's desired format or converted to grayscale), or additional augmentations to make your model generalize better. This particular dataset woul

In [ ]:
with open("Hard-Hat-Universe-26/data.yaml") as stream:
    try:
        print(yaml.safe_load(stream)["names"])
    except yaml.YAMLError as exc:
        print(exc)

['head', 'helmet', 'hi-viz helmet', 'hi-viz vest', 'person']


In [ ]:
file_path = (
    "Hard-Hat-Universe-26/test/labels/"
    "005304_jpg.rf.dc73a5b2a1b38be45392309595f0bae3.txt"
)

with open(file_path, "r") as f:
    print(f.read())

1 0.43866666666666665 0.472 0.264 0.24
1 0.18266666666666667 0.437 0.312 0.274
1 0.7906666666666666 0.457 0.38666666666666666 0.338
1 0.712 0.351 0.336 0.306
4 0.21466666666666667 0.658 0.32266666666666666 0.68
4 0.43866666666666665 0.682 0.22133333333333333 0.608
4 0.7493333333333333 0.659 0.496 0.674
4 0.6333333333333333 0.565 0.34933333333333333 0.694


# Dataset cleaning

In [ ]:
!mkdir datasets
!mv Hard-Hat-Universe-26 datasets/Hard-Hat-Universe-26

In [ ]:
def filter_labels(label_path):
    """
    Rewrites annotation files and filters out any classes
    that are not 0 (head) or 1 (helmet).

    Args:
        label_path (str): A path to the txt annotation file.

    Returns:
        bool: Returns True if resulting file contains at least 1 label.
    """
    new_lines = []
    with open(label_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            label_index = int(line.split()[0])
            if label_index in {0, 1}:
                new_lines.append(line)

    with open(label_path, "w") as file:
        file.writelines(new_lines)

    return len(new_lines) > 0


dataset_dir = "datasets/Hard-Hat-Universe-26"
removed_files_cnt = 0

for split in ["train", "test", "valid"]:
    images_dir = os.path.join(dataset_dir, split, "images")
    labels_dir = os.path.join(dataset_dir, split, "labels")

    for label_file in os.listdir(labels_dir):
        label_path = os.path.join(labels_dir, label_file)
        image_path = os.path.join(images_dir, label_file.replace(".txt", ".jpg"))

        bboxes_exist = filter_labels(label_path)
        if not bboxes_exist:
            # Delete files if there are no bboxes
            os.remove(label_path)
            os.remove(image_path)
            removed_files_cnt += 1

print(f"Deleted {removed_files_cnt} files")

Deleted 1 files


# Training

In [ ]:
config = {
    "train": "Hard-Hat-Universe-26/train/images",
    "val": "Hard-Hat-Universe-26/valid/images",
    "test": "Hard-Hat-Universe-26/test/images",
    "nc": 2,
    "names": ["head", "helmet"]
}

with open("yolov8_config.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

In [ ]:
model = YOLO("yolov8n.pt")
model.train(data="yolov8_config.yaml", epochs=70)

New https://pypi.org/project/ultralytics/8.2.64 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolov8_config.yaml, epochs=70, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nm

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7988e6a97250>
fitness: 0.7146550101618692
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.67976,     0.69348])
names: {0: 'head', 1: 'helmet'}
plot: True
results_dict: {'metrics/precision(B)': 0.9436547591868141, 'metrics/recall(B)': 0.9223755765901628, 'metrics/mAP50(B)': 0.9669594604461744, 'metrics/mAP50-95(B)': 0.6866211823525019, 'fitness': 0.7146550101618692}
save_dir: PosixPath('runs/detect/train2')
speed: {'preprocess': 0.20671093143533278, 'inference': 1.9575200627177238, 'loss': 0.000611390217696086, 'postprocess': 1.6500782932891198}

# Prediction

In [ ]:
model_trained = YOLO("/content/runs/detect/train2/weights/best.pt")

In [ ]:
def draw_boxes(frame, detections, class_counts, class_colors):
    """
    Draws bboxes and class counter.

    Args:
        frame (np.ndarray): Input frame.
        detections (list[tuple[torch.Tensor]]): Predicted bboxes parameters.
        class_counts (list[int]): Class counter on the input frame.
        class_colors (dict[int: tuple[int]]): Class colormap.

    Returns:
        np.ndarray: Resulting frame with bboxes.
    """
    class_names = {
        "head": "person",
        "helmet": "helmet"
    }

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        label = f"{model_trained.names[int(cls)]} {conf:.2f}"
        label_name = label.split()[0]
        label_conf = label.split()[1]
        color = class_colors[int(cls)]
        # bbox
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        # text background
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
        cv2.rectangle(frame,
                      (int(x1), int(y1) - h - 10), (int(x1) + w, int(y1)),
                      color, -1)
        # text
        cv2.putText(frame, f"{class_names[label_name]} {label_conf}",
                    (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    overlay = frame.copy()
    alpha = 0.6
    cv2.rectangle(overlay, (0, 0), (180, 70), (0, 0, 0), -1)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    cv2.putText(frame, f"Person: {class_counts[0]}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, class_colors[0], 2)
    cv2.putText(frame, f"Helmet: {class_counts[1]}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, class_colors[1], 2)

    return frame


input_video_path = "workers_1.mp4"
output_video_path = "workers_1_annotated.mp4"
cap = cv2.VideoCapture(input_video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

class_colormap = {
    0: (242, 174, 113),
    1: (69, 86, 234)
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model_trained(frame, conf=0.4)
    class_counts = [0, 0]  # [head_count, helmet_count]
    detections = []
    for result in results:
        for *box, conf, cls in result.boxes.data:
            detections.append((*box, conf, cls))
            class_counts[int(cls)] += 1

    frame = draw_boxes(frame, detections, class_counts, class_colormap)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 head, 3 helmets, 24.6ms
Speed: 3.3ms preprocess, 24.6ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 head, 3 helmets, 11.4ms
Speed: 3.1ms preprocess, 11.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 head, 3 helmets, 9.7ms
Speed: 3.4ms preprocess, 9.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 head, 4 helmets, 21.0ms
Speed: 5.4ms preprocess, 21.0ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 helmets, 8.7ms
Speed: 2.7ms preprocess, 8.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 helmets, 7.8ms
Speed: 2.9ms preprocess, 7.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 helmets, 8.5ms
Speed: 2.9ms preprocess, 8.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 helmets, 9.3ms
Speed: 3.4ms preprocess, 9.3ms inference, 2.3ms postproces